# Import

In [1]:
import copy
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms.v2 import (
    ToTensor,
    RandomHorizontalFlip,
    Compose,
    RandomResizedCrop,
    RandomShortestSize,
    AutoAugment,
)

from torchvision.transforms.autoaugment import AutoAugmentPolicy
from fvcore.nn import FlopCountAnalysis, flop_count_table
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import os

####################################################
from src.Mymodel import Block
from src.Mymodel import MyResNet34
from src.Mymodel import MyResNet_CIFAR
from src.Mydataloader import LoadDataset

# Setup

In [2]:
"""Dataset selection"""
DATASET = "CIFAR10"
# DATASET = "CIFAR100"
# DATASET = "ImageNet2012"

"""Model selection" for CIFAR"""
num_layer_factor = 5


"""Dataset parameters"""
BATCH = 128
SHUFFLE = True
NUMOFWORKERS = 8
PIN_MEMORY = True

"""optimizer parameters"""
# OPTIMIZER = "SGD"
OPTIMIZER = "Adam"


USE_AMP = True
LOAD_BEFORE_TRAINING = False
# LOAD_BEFORE_TRAINING = True
NUM_EPOCHS = 5000

# Loading the dataset

## Define Dateloader

In [3]:
from src.Mydataloader import LoadDataset

## Confirm that the dataset is loaded properly

In [4]:
_dataset = LoadDataset(root="data", seceted_dataset=DATASET)
train_data, valid_data, test_data, COUNT_OF_CLASSES = _dataset.Unpack()

train_dataloader = DataLoader(
    train_data,
    batch_size=BATCH,
    shuffle=SHUFFLE,
    num_workers=NUMOFWORKERS,
    pin_memory=PIN_MEMORY,
    # pin_memory_device="cuda",
    persistent_workers=True,
)
valid_dataloader = DataLoader(
    valid_data,
    batch_size=BATCH,
    shuffle=SHUFFLE,
    num_workers=NUMOFWORKERS,
    pin_memory=PIN_MEMORY,
    # pin_memory_device="cuda",
    persistent_workers=True,
)
test_dataloader = DataLoader(
    test_data,
    batch_size=BATCH,
    shuffle=SHUFFLE,
    num_workers=NUMOFWORKERS,
    pin_memory=PIN_MEMORY,
    # pin_memory_device="cuda",
    persistent_workers=True,
)

print("train.transforms =", train_data.transform)
print("valid.transforms =", valid_data.transform)
print("test.transforms =", test_data.transform)

/home/lee/anaconda3/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


-----------------------------------------------------------------------
Dataset :  CIFAR10
- Length of Train Set :  45000
- Length of Valid Set :  5000
- Length of Test Set :  10000
- Count of Classes :  10
-----------------------------------------------------------------------
train.transforms = Compose(
      ToTensor()
      Submean()
      AutoAugment(interpolation=InterpolationMode.NEAREST, policy=AutoAugmentPolicy.CIFAR10)
      RandomHorizontalFlip(p=0.5)
)
valid.transforms = Compose(
      ToTensor()
      Submean()
)
test.transforms = Compose(
      ToTensor()
      Submean()
)


In [5]:
print(train_dataloader.batch_size)
print(valid_dataloader.batch_size)
print(test_dataloader.batch_size)

128
128
128


In [6]:
if DATASET != "ImageNet2012":
    for X, y in test_dataloader:
        print(f"Shape of X [N, C, H, W]: {X.shape}")
        print("mean of X", X.mean(dim=(0, 2, 3)))
        print(f"Shape of y: {y.shape} {y.dtype}")
        break

Shape of X [N, C, H, W]: torch.Size([128, 3, 32, 32])
mean of X tensor([-4.8158e-09, -2.6955e-09, -5.6632e-09])
Shape of y: torch.Size([128]) torch.int64


In [7]:
if DATASET != "ImageNet2012":
    class_names = test_dataloader.dataset.classes
    count = 0
    fig, axs = plt.subplots(2, 5, figsize=(8, 4))

    for images, labels in test_dataloader:
        images = images.numpy()

        for i in range(len(images)):
            image = images[i]
            label = labels[i]
            image = np.transpose(image, (1, 2, 0))
            image = np.clip(image, 0, 1)
            ax = axs[count // 5, count % 5]
            ax.imshow(image)
            ax.set_title(f"{class_names[label], label}")
            ax.axis('off')
            count += 1
            
            if count == 10:
                break
        if count == 10:
            break
    plt.tight_layout()
    plt.show()

# Define ResNet

In [8]:
from src.Mymodel import MyResNet34
from src.Mymodel import MyResNet_CIFAR

## Model Confirm

In [9]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [10]:
if DATASET == "CIFAR10" or DATASET == "CIFAR100":
    """ResNet{20, 32, 44, 56, 110, 1202} for CIFAR"""
    model = MyResNet_CIFAR(num_classes=COUNT_OF_CLASSES, num_layer_factor=5).to(device)
    print(f"ResNet-{5*6+2} for {DATASET} is loaded.")
    
elif DATASET == "ImageNet2012":
    """ResNet34 for ImageNet 2012"""
    model = MyResNet34(num_classes=COUNT_OF_CLASSES).to(device)
    # model = models.resnet34(pretrained=True).to(device)
    # model = models.resnet34(pretrained=False).to(device)
    print(f"ResNet-34 for {DATASET} is loaded.")
    
# model.named_modules

ResNet-32 for CIFAR10 is loaded.


In [11]:
tmp_input = torch.rand(BATCH, 3, 32, 32).to(device)
flops = FlopCountAnalysis(model, tmp_input)
print(flop_count_table(flops))

| module                   | #parameters or shape   | #flops     |
|:-------------------------|:-----------------------|:-----------|
| model                    | 0.464M                 | 8.998G     |
|  conv32blocks            |  23.792K               |  3.181G    |
|   conv32blocks.0         |   0.432K               |   56.623M  |
|    conv32blocks.0.weight |    (16, 3, 3, 3)       |            |
|   conv32blocks.1         |   4.672K               |   0.625G   |
|    conv32blocks.1.conv1  |    2.304K              |    0.302G  |
|    conv32blocks.1.bn1    |    32                  |    10.486M |
|    conv32blocks.1.conv2  |    2.304K              |    0.302G  |
|    conv32blocks.1.bn2    |    32                  |    10.486M |
|   conv32blocks.2         |   4.672K               |   0.625G   |
|    conv32blocks.2.conv1  |    2.304K              |    0.302G  |
|    conv32blocks.2.bn1    |    32                  |    10.486M |
|    conv32blocks.2.conv2  |    2.304K              |    0.302

# Define Training

## Define Optimazer
#### ResNet32 for CIFAR10
- SGD
- Batch size = 128 * 2 GPU
- lr : 0.1 -> 0.01 -> 0.001 (at 32k, 48k and terminate on 64k iter)
  - divided by 10 when the error plateaus
- Weight Decay = 0.0001
- Momentum = 0.9
#### ResNet34 for ImageNet2012
- SGD
- Batch size = 256
- lr = 0.1
  - divided by 10 when the error plateaus
  - amount 60k iter
- Weight Decay = 0.0001
- Momentum = 0.9

In [12]:
criterion = nn.CrossEntropyLoss()

if OPTIMIZER == "Adam":
    optimizer = torch.optim.Adam(model.parameters())
    # optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-4)    
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=1e-4)
elif OPTIMIZER == "SGD":
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)

## Define Early Stopping

In [13]:
# Early stopping 관련 변수 초기화
best_val_loss = float("inf")
patience = 1200  # 몇 번까지 기다릴 것인지
early_stop_counter = 0

## Define Learning Rate schedualer

In [14]:
scheduler_mapping = {"CIFAR100": 1000, "CIFAR10": 1000, "ImageNet2012": 30}
MIN_LR = 0.0001
scheduler = ReduceLROnPlateau(
    optimizer,
    mode="min",
    patience=scheduler_mapping[DATASET],
    factor=0.1,
    verbose=True,
    threshold=1e-4,
    min_lr=MIN_LR,
)

## Load before process

In [15]:
scaler = torch.cuda.amp.GradScaler(enabled=True)
file_path = ""
if DATASET == "ImageNet2012":
    file_path = f"{DATASET}/MyResNet34_{BATCH}_{OPTIMIZER}"
else:
    file_path = f"{DATASET}/MyResNet{num_layer_factor*6+2}_{BATCH}_{OPTIMIZER}"

if LOAD_BEFORE_TRAINING == True:
    if os.path.exists("logs/" + file_path + ".pth.tar"):
        # Read checkpoint as desired, e.g.,
        checkpoint = torch.load(
            "logs/" + file_path + ".pth.tar",
            map_location=lambda storage, loc: storage.cuda(device),
        )
        model.load_state_dict(checkpoint["model"])
        optimizer.load_state_dict(checkpoint["optimizer"])
        scaler.load_state_dict(checkpoint["scaler"])
        logs = checkpoint["logs"]

        print("Suceessfully loaded the All setting and Log file.")
        print(file_path)
    else:
        raise ValueError("File does not exist. Check the file path.")
        

else:
    # Create a dictionary to store the variables
    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []
    test_loss = []
    test_acc = []

    logs = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "valid_loss": valid_loss,
        "valid_acc": valid_acc,
        "test_loss": test_loss,
        "test_acc": test_acc,
    }
    print("File does not exist. Created a new log.")

File does not exist. Created a new log.


# [Training Loop]

In [16]:
for epoch in range(NUM_EPOCHS):
    print(f"[Epoch {epoch+1}/{NUM_EPOCHS}] :")
    # Training loop @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm.tqdm(train_dataloader):
        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=USE_AMP):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    running_loss += loss.item()
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_dataloader)
    train_acc = correct / total

    # Evaluation loop @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in valid_dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    valid_loss /= len(valid_dataloader)
    valid_acc = correct / total

    # Testing loop @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    if DATASET == "ImageNet2012":
        pass
    else:
        model.eval()
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in test_dataloader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        test_loss /= len(test_dataloader)
        test_acc = correct / total

    # Print epoch statistics
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
    logs["train_loss"].append(train_loss)
    logs["train_acc"].append(train_acc)

    print(f"Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc*100:.2f}%")
    logs["valid_loss"].append(valid_loss)
    logs["valid_acc"].append(valid_acc)

    if DATASET == "ImageNet2012":
        pass
    else:
        print(f"Test  Loss: {test_loss:.4f} | Test Acc: {test_acc*100:.2f}%")
        logs["test_loss"].append(test_loss)
        logs["test_acc"].append(test_acc)

    ######################################################################
    checkpoint = {
    "model": model.state_dict(),
    "optimizer": optimizer.state_dict(),
    "scaler": scaler.state_dict(),
    "logs": logs,
    }
    torch.save(checkpoint, "logs/" + file_path + ".pth.tar")
    print(f"Saved PyTorch Model State to [logs/{file_path}.pth.tar]")
    ######################################################################
    # Early stopping check
    scheduler.step(valid_loss)
    if valid_loss < best_val_loss:
        best_val_loss = valid_loss
        early_stop_counter = 0

        torch.save(model.state_dict(), "models/" + file_path + ".pth")

    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping after {epoch} epochs without improvement.")
            break

    print("-" * 50)

[Epoch 1/5000] :


100%|██████████| 352/352 [00:10<00:00, 33.64it/s]


Train Loss: 0.0032 | Train Acc: 61.11%
Valid Loss: 1.3778 | Valid Acc: 52.26%
Test  Loss: 1.3452 | Test Acc: 53.35%
--------------------------------------------------
[Epoch 2/5000] :


  7%|▋         | 25/352 [00:00<00:10, 32.46it/s]


KeyboardInterrupt: 

## Training Results

In [20]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 5))

# 첫 번째 그래프: Training and Test Loss
axs[0].plot(logs["train_loss"], label="Training Loss")
axs[0].plot(logs["valid_loss"], label="Validation Loss")
axs[0].plot(logs["test_loss"], label="Test Loss")
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Loss")
axs[0].set_title("Training, Validation and Test Loss")
axs[0].legend()

# 두 번째 그래프: Training and Test Accuracy
axs[1].plot(logs["train_acc"], label="Training Accuracy")
axs[1].plot(logs["valid_acc"], label="Validation Accuracy")
axs[1].plot(logs["test_acc"], label="Test Accuracy")
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Accuracy")
axs[1].set_title("Training, Validation and Test Accuracy")
axs[1].legend()

# 그래프를 보여줍니다.
plt.tight_layout()
plt.show()

주피터노트북 output set
@tag:notebookOutputLayout